<a href="https://colab.research.google.com/github/shiehn/dawnet-remotes/blob/main/DAWNet_Remote_BeatNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

INSTALL DEPENDENCIES

In [5]:
#Check if librosa is installed, if not then install it
try:
    import librosa
except Exception:
    !pip install librosa
    import librosa

#Check if madmom is installed, if not then install it
try:
    import madmom
except Exception:
    !pip install --upgrade --no-deps --force-reinstall --quiet 'git+https://github.com/CPJKU/madmom.git'
    import madmom

#Check if pyaudio is installed, if not then install it
try:
    import pyaudio
except Exception:
    !apt-get install portaudio19-dev
    !pip install pyaudio
    import pyaudio

#Check if BeatNet is installed, if not then install it
try:
    from BeatNet.BeatNet import BeatNet
except Exception:
    !pip install BeatNet
    from BeatNet.BeatNet import BeatNet


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libportaudio2 libportaudiocpp0
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 30 not upgraded.
Need to get 188 kB of archives.
After this operation, 927 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudiocpp0 amd64 19.6.0-1.1 [16.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 portaudio19-dev amd64 19.6.0-1.1 [106 kB]
Fetched 188 kB in 1s (282 kB/s)
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 121658 files and directories currently installed.)
Preparing to unpack .../libportaudio2_19.6.0-1.

In [16]:
# Check if ffmpeg is installed, if not then install it
ffmpeg_installed = !command -v ffmpeg
if not ffmpeg_installed:
    !apt-get install ffmpeg

#Check if dawnet-client is installed, if not then install it
try:
    import dawnet_client
except ImportError:
    !pip install dawnet-client
    import dawnet_client

try:
    import dawnet_client
except ImportError:
  !pip install dawnet-client --upgrade


import dawnet_client.core as dawnet
from dawnet_client import DAWNetFilePath
import os
import shutil
import uuid
import numpy as np

placeholder_txt = "Enter the token generated by the DAWNet plugin"
DAWNET_TOKEN = "24a3a56f-f1c3-4a07-b68f-2cdacb8a61d8" #@param {type:"string"}
dawnet_token = DAWNET_TOKEN

if dawnet_token is None or dawnet_token == "" or dawnet_token == placeholder_txt:
  print("ERROR: The token provided is not valid.")
  exit()

def calculate_average_bpm(output):
    """
    Calculate the average BPM from BeatNet output.

    :param output: numpy array of shape (n, 2) where each row is [time, beat_position]
    :return: Average BPM as a float
    """
    # Filter to get the first beat of each bar (assuming it's marked as 1)
    first_beats = output[output[:, 1] == 1]

    # Calculate time differences between successive first beats
    time_differences = np.diff(first_beats[:, 0])

    # Calculate average time difference
    if len(time_differences) == 0:
        return 0  # Avoid division by zero if there are no beats detected
    average_time_diff = np.mean(time_differences)

    # Convert average time difference to BPM
    bpm = 60 / average_time_diff
    return bpm

# Example usage
output = np.array([...])  # Replace with your BeatNet output
average_bpm = calculate_average_bpm(output)
print("Average BPM:", average_bpm)

async def dawnet_func(input_file: DAWNetFilePath):
    try:
      # Generate a unique directory name
      unique_dir_name = f"/tmp/{uuid.uuid4()}"
      os.makedirs(unique_dir_name, exist_ok=True)

      # Copy the input file to the new directory
      new_file_path = os.path.join(unique_dir_name, os.path.basename(input_file))
      shutil.copy(input_file, new_file_path)

      # Initialize the estimator
      estimator = BeatNet(1, mode='offline', inference_model='DBN', plot=[], thread=False)

      # Use the new directory in estimator.process
      Output = estimator.process(new_file_path)

      # Convert to a numpy array
      output_array = np.array(Output)

      bpm = calculate_average_bpm(output_array)

      # after executing your custom code you send data back to the plugin like so ..
      await dawnet.output().add_file(input_file)
      await dawnet.output().add_log(f"BPM is: {bpm}")
      await dawnet.output().add_message(f"BPM is: {bpm}")
      await dawnet.output().send()

      return True
    except Exception as e:
      await dawnet.output().add_error(f"Error in arbitrary_method: {e}")

      return False


dawnet.set_token(token=dawnet_token)
dawnet.set_name("DAWNet Template")
dawnet.set_description("This is a template for creating a DAWNet Remote")
dawnet.register_method(dawnet_func)

dawnet.connect_to_server()

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed